In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [5]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.LM(
    model.parameters(), lr=1, mu=0.001, mu_dec=0.1, model=model, use_diagonal=False, c1=1e-4, tau=0.1, line_search_method="backtrack"
)

times = []

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.06667272746562958
epoch:  1, loss: 0.060904838144779205
epoch:  2, loss: 0.055892251431941986
epoch:  3, loss: 0.05172598361968994
epoch:  4, loss: 0.047193098813295364
epoch:  5, loss: 0.042146582156419754
epoch:  6, loss: 0.03771459311246872
epoch:  7, loss: 0.033440738916397095
epoch:  8, loss: 0.029551977291703224
epoch:  9, loss: 0.026136722415685654
epoch:  10, loss: 0.02308354154229164
epoch:  11, loss: 0.020622970536351204
epoch:  12, loss: 0.018363410606980324
epoch:  13, loss: 0.016537850722670555
epoch:  14, loss: 0.014877413399517536
epoch:  15, loss: 0.013560688123106956
epoch:  16, loss: 0.01230364665389061
epoch:  17, loss: 0.01119944453239441
epoch:  18, loss: 0.010232573375105858
epoch:  19, loss: 0.009532867930829525
epoch:  20, loss: 0.008966640569269657
epoch:  21, loss: 0.008453687652945518
epoch:  22, loss: 0.008056050166487694
epoch:  23, loss: 0.007768329232931137
epoch:  24, loss: 0.007491347845643759
epoch:  25, loss: 0.007256527431309223
ep

In [6]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.9247288903203285
Test metrics:  R2 = 0.8826490323424914


In [7]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.LM(
    model.parameters(), lr=1, mu=0.001, mu_dec=0.1, model=model, use_diagonal=True, c1=1e-4, tau=0.1, line_search_method="backtrack"
)

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.33193057775497437
epoch:  1, loss: 0.3225981891155243
epoch:  2, loss: 0.3110601603984833
epoch:  3, loss: 0.29986247420310974
epoch:  4, loss: 0.28774046897888184
epoch:  5, loss: 0.28667518496513367
epoch:  6, loss: 0.2768465280532837
epoch:  7, loss: 0.25803226232528687
epoch:  8, loss: 0.2483709752559662
epoch:  9, loss: 0.23762786388397217
epoch:  10, loss: 0.22619642317295074
epoch:  11, loss: 0.21426819264888763
epoch:  12, loss: 0.20327989757061005
epoch:  13, loss: 0.19252778589725494
epoch:  14, loss: 0.18175937235355377
epoch:  15, loss: 0.1711900383234024
epoch:  16, loss: 0.16081362962722778
epoch:  17, loss: 0.15149827301502228
epoch:  18, loss: 0.14149236679077148
epoch:  19, loss: 0.13192133605480194
epoch:  20, loss: 0.12253682315349579
epoch:  21, loss: 0.11362215876579285
epoch:  22, loss: 0.10550106316804886
epoch:  23, loss: 0.09726785868406296
epoch:  24, loss: 0.09023772925138474
epoch:  25, loss: 0.08311010152101517
epoch:  26, loss: 0.0767146

In [8]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.9868969644175797
Test metrics:  R2 = 0.9614377460292693
